In [ ]:
import os
import importlib  # used to reload modules if necessary
path_to_frontend = '..'
os.chdir(path_to_frontend)
print("current working directory: " + os.getcwd())
import q100viz.session as session
import q100viz.devtools as devtools
import q100viz.graphics.graphs as graphs

## Simulation

In [ ]:
# prepare reference data: select buildings without decisions
session.buildings.df['selected'] = True
session.buildings.df['group'] = 1

In [ ]:
# simulate until 2045
session.simulation.setup(input_max_year=2045, export_graphs=False)
session.simulation.run()

## Export gray individual graphs:
### individual emissions

In [ ]:
########## individual buildings data ########
for idx in session.buildings.df.index:

    # export emissions graph:
    graphs.export_individual_emissions(
        csv_name="/emissions/CO2_emissions_{0}.csv".format(
            idx),
        data_folders=session.simulation.output_folders,
        columns=['building_household_emissions'],
        title_="Emissionen",
        outfile=session.simulation.current_output_folder +
        "/emissions/CO2_emissions_{0}.png".format(idx),
        xlabel_="Jahr",
        ylabel_="$CO_{2}$-Äquivalente (kg/Monat)",  # TODO: t/Jahr
        x_='current_date',
        convert_grams_to_kg=True,
        # compare_data_folder=session.simulation.current_output_folder + "/../../precomputed/simulation_defaults",
        figtext=
            str(idx) + " "
            + str(group_df.loc[idx, 'address']) + " "
            + str(group_df.loc[idx, 'type'])
            + "\nø-spez. Wärmeverbrauch: "
            + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
            + ", ø-spez. Stromverbrauch: "
            + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
            if session.VERBOSE_MODE else "",
        figsize=(16,12),  # inches
        overwrite_color='lightgray',
        show_legend=False
    )

### individual energy costs

In [ ]:
for idx in session.buildings.df.index:
    # export energy prices graph:
    graphs.export_individual_energy_expenses(
        building_idx=idx,
        csv_name="/energy_prices/energy_prices_{0}.csv".format(
            idx),
        data_folders=session.simulation.output_folders,
        columns=['building_household_expenses_heat',
                'building_household_expenses_power'],
        labels_=['Wärmekosten', 'Stromkosten'],
        outfile=session.simulation.current_output_folder +
        "/energy_prices/energy_prices_{0}.png".format(idx),
        title_="Energiekosten",
        xlabel_="Jahr",
        ylabel_="€/Monat",
        x_='current_date',
        figtext=
            str(idx) + " "
            + str(group_df.loc[idx, 'address']) + " "
            + str(group_df.loc[idx, 'type'])
            + "\nø-spez. Wärmeverbrauch: "
            + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
            + ", ø-spez. Stromverbrauch: "
            + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
            if session.VERBOSE_MODE else "",
        figsize=(16,12),  # inches
        overwrite_color='lightgray',
        label_show_iteration_round=False,
        prepend_historic_data=True
    )

## Copy data

after the simulation is done, copy the freshly created folder to `..data/precomputed/simulation_defaults`

In [ ]:
import shutil

source = session.simulation.current_output_folder
destination = session.simulation.reference_data_folder

shutil.copytree(source, destination, dirs_exist_ok=True)